In [58]:
import os
import time
from selenium.webdriver import Firefox, FirefoxOptions, FirefoxService
import pandas as pd

In [59]:
from selenium.webdriver.support.select import Select

In [60]:
options = FirefoxOptions()
options.profile = 'C:\\Users\\leowy\\AppData\\Roaming\\Mozilla\\Firefox\\Profiles\\zqu1wxis.default-release-1683035901305'
service = FirefoxService(executable_path='C:\\Users\\leowy\\anaconda3\\envs\\market-returns\\Scripts\\geckodriver.exe')

C:\Users\leowy\AppData\Local\Temp\ipykernel_972\1079762744.py:2: DeprecationWarning: Setting a profile has been deprecated. Please use the set_preference and install_addons methods
  options.profile = 'C:\\Users\\leowy\\AppData\\Roaming\\Mozilla\\Firefox\\Profiles\\zqu1wxis.default-release-1683035901305'
C:\Users\leowy\AppData\Local\Temp\ipykernel_972\1079762744.py:3: DeprecationWarning: Firefox will soon stop logging to geckodriver.log by default; Specify desired logs with log_output
  service = FirefoxService(executable_path='C:\\Users\\leowy\\anaconda3\\envs\\market-returns\\Scripts\\geckodriver.exe')


In [61]:
driver = Firefox(options=options, service=service)
driver.implicitly_wait(0.5)

In [62]:
driver.get('https://markets.ft.com/data/indices/tearsheet/historical?s=TFTSTIU:FSI')

In [63]:
driver.find_element(by='class name', value='mod-ui-filter-overlay__filter-toggle').click()

In [64]:
def select_start_date(driver: Firefox, year: int):
    driver.find_elements(by='class name', value='picker__input')[0].click()
    
    if year == 2008:
        start_year = driver.find_element(by='xpath', value='/html/body/div[3]/div[3]/section/div[1]/div/div/div[1]/div[1]/div/form/fieldset/span/div[1]/div[2]/div/div/div/div/div/div/select')
        start_year_select = Select(start_year)
        start_year.click()
        start_year_select.select_by_index(0)
        
    start_year = driver.find_element(by='xpath', value='/html/body/div[3]/div[3]/section/div[1]/div/div/div[1]/div[1]/div/form/fieldset/span/div[1]/div[2]/div/div/div/div/div/div/select')
    start_year_select = Select(start_year)
    start_year.click()
    start_year_select.select_by_value(f'{year}')
    
    for i in range(15):
        start_month = driver.find_element(by='xpath', value='/html/body/div[3]/div[3]/section/div[1]/div/div/div[1]/div[1]/div/form/fieldset/span/div[1]/div[2]/div/div/div/div/div/div/div[1]')
        if start_month.text == 'January':
            break
        start_prev = driver.find_element(by='xpath', value='/html/body/div[3]/div[3]/section/div[1]/div/div/div[1]/div[1]/div/form/fieldset/span/div[1]/div[2]/div/div/div/div/div/div/div[2]')
        start_prev.click()
        
    start_date_cell = driver.find_element(by='css selector', value=f'div[aria-label="1 Jan, {year}"]')
    start_date_cell.click()

In [65]:
def select_end_date(driver: Firefox, year: int):
    end_date_field = driver.find_elements(by='class name', value='picker__input')[1]
    end_date_field.click()
    
    if year == 2008:
        end_year = driver.find_element(by='xpath', value='/html/body/div[3]/div[3]/section/div[1]/div/div/div[1]/div[1]/div/form/fieldset/span/div[2]/div[2]/div/div/div/div/div/div/select')
        end_year_select = Select(end_year)
        end_year.click()
        end_year_select.select_by_index(0)
    
    end_year = driver.find_element(by='xpath', value='/html/body/div[3]/div[3]/section/div[1]/div/div/div[1]/div[1]/div/form/fieldset/span/div[2]/div[2]/div/div/div/div/div/div/select')
    end_year_select = Select(end_year)
    end_year.click()
    end_year_select.select_by_value(f'{year}')
    
    if not year == pd.to_datetime('today').year:
        for i in range(15):
            end_month = driver.find_element(by='xpath', value='/html/body/div[3]/div[3]/section/div[1]/div/div/div[1]/div[1]/div/form/fieldset/span/div[2]/div[2]/div/div/div/div/div/div/div[1]')
            if end_month.text == 'December':
                break
            end_next = driver.find_element(by='xpath', value='/html/body/div[3]/div[3]/section/div[1]/div/div/div[1]/div[1]/div/form/fieldset/span/div[2]/div[2]/div/div/div/div/div/div/div[3]')
            end_next.click()
            
        end_date_cell = driver.find_element(by='css selector', value=f'div[aria-label="31 Dec, {year}"]')
        end_date_cell.click()
    
    else:
        for i in range(15):
            end_month = driver.find_element(by='xpath', value='/html/body/div[3]/div[3]/section/div[1]/div/div/div[1]/div[1]/div/form/fieldset/span/div[2]/div[2]/div/div/div/div/div/div/div[1]')
            if end_month.text == pd.to_datetime('today').strftime('%B'):
                break
            end_next = driver.find_element(by='xpath', value='/html/body/div[3]/div[3]/section/div[1]/div/div/div[1]/div[1]/div/form/fieldset/span/div[2]/div[2]/div/div/div/div/div/div/div[3]')
            end_next.click()
            
        end_date_cell = driver.find_element(by='css selector', value=f'div[aria-label="{pd.to_datetime("today").strftime("%d %b, %Y")}"]')
        end_date_cell.click()

In [66]:
rows = []

for year in range(2008, pd.to_datetime('today').year + 1):
    
    if year == 2008:
        select_start_date(driver, year)
        select_end_date(driver, year)
    
    else:
        select_end_date(driver, year)
        select_start_date(driver, year)
        
    time.sleep(2.5)
    
    rows.extend(driver.find_element(by='class name', value='mod-tearsheet-historical-prices__results').text.split('\n')[:0:-1])

In [67]:
sti = pd.DataFrame([row.rsplit(' ', maxsplit=5) for row in rows[1:]], columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])

In [68]:
sti['Date'] = sti['Date'].apply(pd.to_datetime)

In [69]:
sti = sti.replace(',','', regex=True)

In [70]:
sti[['Open', 'High', 'Low', 'Close', 'Volume']] = sti[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)

In [71]:
sti.set_index('Date', drop=True).to_csv('data/Straits Times Index USD Gross.csv')

In [72]:
driver.quit()